In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> # COM 526 P ANALYTICS & SYSTEMS OF BIG DATA PRACTICE – PROBLEM SET V

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import Binarizer, LabelEncoder
import numpy as np

# Load datasets
avocado_df = pd.read_csv('/kaggle/input/dataset/Avocado Dataset.csv')
trail_df = pd.read_csv('/kaggle/input/dataset/Trail.csv - Trail.csv.csv')

# 1. Select a subset of relevant attributes from Avocado Dataset (PLU 4046, 4225, 4770)
filtered_avocado_df = avocado_df[
    (avocado_df['4046'] > 0) | (avocado_df['4225'] > 0) | (avocado_df['4770'] > 0) & (avocado_df['type'] == 'organic')
]
relevant_columns = ['Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770', 'type', 'year', 'region']
filtered_avocado_relevant_df = filtered_avocado_df[relevant_columns]

# Display the first few rows of the filtered Avocado dataset
print("Filtered Avocado Data (PLU 4046, 4225, 4770, Organic):")
print(filtered_avocado_relevant_df.head())

Filtered Avocado Data (PLU 4046, 4225, 4770, Organic):
         Date AveragePrice  Total Volume     4046       4225    4770  \
0  27-12-2015         1.33      64236.62  1036.74   54454.85   48.16   
1  20-12-2015         1.35      54876.98   674.28   44638.81   58.33   
2  13-12-2015         0.93     118220.22   794.70  109149.67  130.50   
3  06-12-2015         1.08      78992.15  1132.00   71976.41   72.58   
4  29-11-2015         1.29      51039.60   941.48   43838.39   75.78   

           type  year  region  
0  conventional  2015  Albany  
1  conventional  2015  Albany  
2  conventional  2015  Albany  
3  conventional  2015  Albany  
4  conventional  2015  Albany  


In [ ]:
# 2. Remove duplicates and fill missing values in "AveragePrice" in the Trail Dataset
trail_initial_size = trail_df.shape[0]
trail_df_cleaned = trail_df.drop_duplicates().copy()
trail_df_cleaned['AveragePrice'] = trail_df_cleaned['AveragePrice'].fillna(1.25)  # Avoiding inplace operation
trail_final_size = trail_df_cleaned.shape[0]

# Print size before and after removing duplicates
print(f"\nInitial Trail Dataset size: {trail_initial_size}")
print(f"Final Trail Dataset size (after removing duplicates): {trail_final_size}")




Initial Trail Dataset size: 202
Final Trail Dataset size (after removing duplicates): 195


In [ ]:
# 3. Binarize the "Year" column in the Avocado Dataset (Threshold: 2016)
binarizer = Binarizer(threshold=2016)
avocado_df['Year_bin'] = binarizer.fit_transform(avocado_df[['year']])

# Display binarized Year column
print("\nBinarized Year Column (Threshold: 2016):")
print(avocado_df[['year', 'Year_bin']].head())



Binarized Year Column (Threshold: 2016):
   year  Year_bin
0  2015         0
1  2015         0
2  2015         0
3  2015         0
4  2015         0


In [ ]:

# 4. Integer Encoding for categorical attributes in the Avocado Dataset
label_encoder = LabelEncoder()
avocado_df['type_encoded'] = label_encoder.fit_transform(avocado_df['type'])

# Display encoded Type column
print("\nInteger Encoded 'Type' Column:")
print(avocado_df[['type', 'type_encoded']].head())


Integer Encoded 'Type' Column:
           type  type_encoded
0  conventional             0
1  conventional             0
2  conventional             0
3  conventional             0
4  conventional             0


In [ ]:
# 5. One-Hot Encoding for the "Region" column in the Avocado Dataset
region_encoded = pd.get_dummies(avocado_df['region'], prefix='Region')
avocado_df = pd.concat([avocado_df, region_encoded], axis=1)

# Display one-hot encoded columns for Region
print("\nOne-Hot Encoded 'Region' Columns:")
print(avocado_df.filter(like='Region_').head())


One-Hot Encoded 'Region' Columns:
   Region_Albany  Region_Atlanta  Region_BaltimoreWashington  Region_Boise  \
0           True           False                       False         False   
1           True           False                       False         False   
2           True           False                       False         False   
3           True           False                       False         False   
4           True           False                       False         False   

   Region_Boston  Region_BuffaloRochester  Region_California  \
0          False                    False              False   
1          False                    False              False   
2          False                    False              False   
3          False                    False              False   
4          False                    False              False   

   Region_Charlotte  Region_Chicago  Region_CincinnatiDayton  ...  \
0             False           False       

In [ ]:
# 6. Ignore tuples with missing values in the Avocado Dataset
avocado_cleaned_df = avocado_df.dropna()

# Display cleaned Avocado dataset (without NaN values)
print("\nAvocado Data after removing rows with missing values:")
print(avocado_cleaned_df.head())



Avocado Data after removing rows with missing values:
         Date AveragePrice  Total Volume     4046       4225    4770  \
0  27-12-2015         1.33      64236.62  1036.74   54454.85   48.16   
1  20-12-2015         1.35      54876.98   674.28   44638.81   58.33   
2  13-12-2015         0.93     118220.22   794.70  109149.67  130.50   
3  06-12-2015         1.08      78992.15  1132.00   71976.41   72.58   
4  29-11-2015         1.29      51039.60   941.48   43838.39   75.78   

   Total Bags  Small Bags  Large Bags  XLarge Bags  ... Region_SouthCarolina  \
0     8696.87     8603.62       93.25          0.0  ...                False   
1     9505.56     9408.07       97.49          0.0  ...                False   
2     8145.35     8042.21      103.14          0.0  ...                False   
3     5811.16     5677.40      133.76          0.0  ...                False   
4     6183.95     5986.26      197.69          0.0  ...                False   

   Region_SouthCentral Region_S

In [ ]:
# 7. Drop the attribute with high nullity in the Avocado Dataset
nullity_threshold = avocado_df.isnull().mean().idxmax()
avocado_df_dropped = avocado_df.drop(columns=[nullity_threshold])

print(f"\nDropped column with highest nullity: {nullity_threshold}")




Dropped column with highest nullity: AveragePrice


In [ ]:
# 8. Statistical Summary of the Avocado Dataset (for numeric columns only)
numeric_columns = avocado_df.select_dtypes(include=[np.number])  # Select only numeric columns
stat_summary = {
    'Dimensions': avocado_df.shape,
    'Most Frequent Values': avocado_df.mode().iloc[0],
    'Data Types': avocado_df.dtypes,
    'Count': avocado_df.count(),
    'Mean': numeric_columns.mean(),
    'Standard Deviation': numeric_columns.std(),
    'Min': numeric_columns.min(),
    'Max': numeric_columns.max(),
    '25%': numeric_columns.quantile(0.25),
    'Median (50%)': numeric_columns.median(),
    '75%': numeric_columns.quantile(0.75),
    'Class Distribution (Type)': avocado_df['type'].value_counts(normalize=True),
    'Correlation Matrix': numeric_columns.corr(),
    'Skewness': numeric_columns.skew(),
}

# Display statistical summary
print("\nStatistical Summary of the Avocado Dataset:")
print(stat_summary)


Statistical Summary of the Avocado Dataset:
{'Dimensions': (18250, 69), 'Most Frequent Values': Date                       18-03-2018
AveragePrice                     1.15
Total Volume                  2038.99
4046                              0.0
4225                              0.0
                              ...    
Region_Syracuse                 False
Region_Tampa                    False
Region_TotalUS                  False
Region_West                     False
Region_WestTexNewMexico         False
Name: 0, Length: 69, dtype: object, 'Data Types': Date                        object
AveragePrice                object
Total Volume               float64
4046                       float64
4225                       float64
                            ...   
Region_Syracuse               bool
Region_Tampa                  bool
Region_TotalUS                bool
Region_West                   bool
Region_WestTexNewMexico       bool
Length: 69, dtype: object, 'Count': Date          

In [ ]:
# 9. Test drive the use of Gini Index, Information Gain, Entropy and other measures that are supported in your platform, performing the role of data selection.
import numpy as np
import pandas as pd
from math import log2

# Example dataset
data = {
    'Feature1': [1, 1, 0, 0, 1, 0, 1, 0],
    'Feature2': [1, 0, 1, 0, 1, 1, 0, 0],
    'Label':    [1, 0, 1, 0, 1, 0, 1, 0]
}

df = pd.DataFrame(data)

# Function to calculate entropy
def entropy(probabilities):
    return -sum([p * log2(p) for p in probabilities if p != 0])

# Function to calculate Gini Index
def gini(probabilities):
    return 1 - sum([p**2 for p in probabilities])

# Function to calculate Information Gain
def information_gain(df, feature, label):
    # Calculate total entropy for the dataset
    total_entropy = entropy(df[label].value_counts(normalize=True))

    # Split the data by the feature and calculate entropy for each subset
    values = df[feature].unique()
    weighted_entropy = 0
    for value in values:
        subset = df[df[feature] == value]
        subset_entropy = entropy(subset[label].value_counts(normalize=True))
        weighted_entropy += len(subset) / len(df) * subset_entropy

    return total_entropy - weighted_entropy

# Apply calculations for each feature
features = ['Feature1', 'Feature2']
results = {}

for feature in features:
    info_gain = information_gain(df, feature, 'Label')
    gini_index = gini(df[feature].value_counts(normalize=True))
    results[feature] = {'Information Gain': info_gain, 'Gini Index': gini_index}

# Display the results
results_df = pd.DataFrame(results).T
results_df


,Information Gain,Gini Index
Feature1,0.188722,0.5
Feature2,0.188722,0.5


In [ ]:
#10. Test drive the implementation support in your platform of choice for data preprocessing phases such as cleaning, selection, transformation, integration in addition to the earlier exercises.
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# --- Sample dataset with missing values and duplicates ---
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Alice'],
    'Age': [25, np.nan, 35, 45, 25],
    'Salary': [50000, 60000, np.nan, 80000, 50000],
    'Gender': ['Female', 'Male', 'Male', 'Male', 'Female']
}

df = pd.DataFrame(data)

# --- Step 1: Data Cleaning ---
# Remove duplicates
df_cleaned = df.drop_duplicates()

# Fill missing values using .loc[] to avoid SettingWithCopyWarning
df_cleaned.loc[:, 'Age'] = df_cleaned['Age'].fillna(df_cleaned['Age'].mean())
df_cleaned.loc[:, 'Salary'] = df_cleaned['Salary'].fillna(df_cleaned['Salary'].median())

# --- Step 2: Feature Selection ---
# For demonstration, we'll use only numerical features ('Age' and 'Salary') for selection
X = df_cleaned[['Age', 'Salary']]
y = np.random.randint(0, 2, len(df_cleaned))  # Example target variable for feature selection

# Select the best features (k=1 for simplicity here)
selector = SelectKBest(score_func=f_classif, k=1)
X_selected = selector.fit_transform(X, y)

# Get selected feature names
selected_features = X.columns[selector.get_support(indices=True)]

# --- Step 3: Data Transformation ---
# Scale the selected numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# One-hot encode the categorical column using updated `sparse_output` parameter
encoder = OneHotEncoder(sparse_output=False)  # Updated argument to sparse_output
X_encoded = encoder.fit_transform(df_cleaned[['Gender']])

# Combine scaled and encoded features into a final dataset
X_preprocessed = np.concatenate([X_scaled, X_encoded], axis=1)

# --- Step 4: Data Integration (Optional, if you have multiple datasets) ---
# Example: Integrating another dataset (Assume we have an additional dataset)
additional_data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'City': ['NYC', 'LA', 'SF', 'NYC']
}

df_additional = pd.DataFrame(additional_data)

# Merge the cleaned dataset with the additional dataset on 'Name'
df_merged = pd.merge(df_cleaned, df_additional, on='Name')

# --- Display Results ---
print("Cleaned DataFrame:")
print(df_cleaned)
print("\nSelected Feature(s):", selected_features)
print("\nPreprocessed Features (Scaled + Encoded):")
print(X_preprocessed)
print("\nMerged DataFrame:")
print(df_merged)


Cleaned DataFrame:
      Name   Age   Salary  Gender
0    Alice  25.0  50000.0  Female
1      Bob  35.0  60000.0    Male
2  Charlie  35.0  60000.0    Male
3    David  45.0  80000.0    Male

Selected Feature(s): Index(['Salary'], dtype='object')

Preprocessed Features (Scaled + Encoded):
[[-1.14707867  1.          0.        ]
 [-0.22941573  0.          1.        ]
 [-0.22941573  0.          1.        ]
 [ 1.60591014  0.          1.        ]]

Merged DataFrame:
      Name   Age   Salary  Gender City
0    Alice  25.0  50000.0  Female  NYC
1      Bob  35.0  60000.0    Male   LA
2  Charlie  35.0  60000.0    Male   SF
3    David  45.0  80000.0    Male  NYC
